In [ ]:
!pip install stumpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.5/176.5 kB 3.9 MB/s eta 0:00:00


In [ ]:
# libraries
#%matplotlib notebook
import os
import pandas as pd
import numpy as np
import json
from scipy.signal import periodogram
from sklearn.metrics import confusion_matrix, average_precision_score
import matplotlib
import seaborn
import matplotlib.dates as md
from matplotlib import pyplot as plt
from google.colab import drive
import stumpy
drive.mount('/content/drive')

Mounted at /content/drive


### Initialize Yahoo A1

In [ ]:
dfsA1 = {}

# Loop through the range 1 to 67 (inclusive) and load data
for i in range(1, 68):
    file_path = f"/content/drive/My Drive/YAHOO/A1/Yahoo_A1real_{i}_data.out"
    dfsA1[i] = pd.read_csv(file_path, sep=",", names=[f"value_{i}", "label"])

### The following function was taken by https://www.kaggle.com/code/joshuaswords/time-series-anomaly-detection-matrix-profiling , where stumpy.stump Matrix Profile function is used

### Stumpy, but finding the anomalies using a static threshold

In [ ]:
def run_matrix_profile_anomaly_detection_yahoo(df, column_name: str, window_size: int = 96, smooth_n: int = 10, normalize_data: bool = True, plot_results: bool = True, threshold_number=95):
    """
    df: Pandas DataFrame
    column_name: Name of the column to analyze
    window_size: Approximately, how many data points might be found in a pattern
    smooth_n: Time windows to smooth matrix profile scores by
    normalize_data: Normalize dataframe or not. Advisable when using columns which are in diff. scales. Boolean
    plot_results: Return a plotly plot of the results. Includes the original timeseries and the matrix profile.
    threshold_number: Percentile threshold for anomaly detection

    Returns:
    Matrix Profile Scores, ranking score for the column, and evaluation metrics (precision, recall, F1-score, PR AUC).
    """
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in the DataFrame.")

    # Select the specified column
    column_data = df[column_name]

    # Normalize data if required
    if normalize_data:
        column_data = (normalize(column_data).dropna())

    # Calculate matrix profile
    mp = stumpy.stump(column_data.astype("float"), m=window_size, ignore_trivial=True, normalize=False)
    mp_dist = mp[:, 0]

    # Smooth the matrix profile scores
    df_mp_dist = pd.DataFrame(mp_dist).rolling(smooth_n).mean()
    score = df_mp_dist[0].quantile(0.995) - df_mp_dist[0].quantile(0.75)

    # Fill NaN values
    nan_value_count = np.empty(len(column_data) - len(mp_dist))
    nan_value_count.fill(np.nan)
    mp_dist = np.concatenate((nan_value_count, mp_dist.astype(float)))

    # Plot the results if required
    if plot_results:
        pd.options.plotting.backend = "plotly"
        df_tmp = df[[column_name]].assign(MatrixProfileScore=mp_dist)
        fig = df_tmp.plot(template="simple_white", title=f"{column_name}")
        fig.show()

    # Evaluate metrics
    ground_truth = df['label'].values

    # Remove NaN values and replace them with 0
    mp_dist[np.isnan(mp_dist)] = 0

    # Apply a threshold for the matrix profile
    threshold = np.percentile(mp_dist, threshold_number)

    # Find the anomalies
    predicted_anomalies = (mp_dist > threshold).astype(int)

    # Compute metrics using sklearn
    precision = precision_score(ground_truth, predicted_anomalies)
    recall = recall_score(ground_truth, predicted_anomalies)
    f1 = f1_score(ground_truth, predicted_anomalies)
    pr_auc = average_precision_score(ground_truth, predicted_anomalies)

    return mp_dist, score, precision, recall, f1, pr_auc


### Yahoo A1A

In [ ]:
from itertools import product
import numpy as np
import pandas as pd
from itertools import product
import warnings
from sklearn.metrics import precision_score, recall_score, f1_score, average_precision_score

# Suppress warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

A1A = [1,2,3,8,9,10,13,23,24,27,34,41,66]

# Initialize global metrics
global_pr_auc_scores = []

# Define hyperparameter ranges
window_sizes = [10, 24, 50, 75, 100, 150]  # Example values; modify as needed
thresholds = [80, 85, 90, 95, 99]  # Percentile thresholds

optimal_results = []

for i in A1A:
    normal_data = dfsA1[i]
    values = normal_data[f"value_{i}"].values
    ground_truth = normal_data['label'].values  # 1 = anomaly, 0 = normal

    best_f1 = 0
    best_params = None
    best_metrics = None

    for window_size, threshold in product(window_sizes, thresholds):
        matrix_profile_distances, score, precision, recall, f1, pr_auc = run_matrix_profile_anomaly_detection_yahoo(
            dfsA1[i],
            f"value_{i}",
            window_size=window_size,
            normalize_data=False,
            smooth_n=(5*30),
            plot_results=False,
            threshold_number=threshold,
        )

        if f1 > best_f1:
            best_f1 = f1
            best_params = (window_size, threshold)
            best_metrics = (precision, recall, pr_auc)

    # Update global metrics with best metrics
    if best_metrics:
        precision, recall, pr_auc = best_metrics
        global_pr_auc_scores.append(pr_auc)

        # Store optimal results for the dataset
        optimal_results.append({
            'dataset': i,
            'best_f1': best_f1,
            'best_window_size': best_params[0],
            'best_threshold': best_params[1],
            'precision': precision,
            'recall': recall,
            'pr_auc': pr_auc
        })

        matrix_profile_distances, score, precision, recall, f1, pr_auc = run_matrix_profile_anomaly_detection_yahoo(
            dfsA1[i],
            f"value_{i}",
            window_size=best_params[0],
            normalize_data=False,
            smooth_n=(5*30),
            plot_results=True,
            threshold_number=best_params[1],
        )

# Compute global metrics
global_recall = np.mean([result['recall'] for result in optimal_results])
global_precision = np.mean([result['precision'] for result in optimal_results])
global_f1 = np.mean([result['best_f1'] for result in optimal_results])
global_pr_auc = np.mean(global_pr_auc_scores)

# Print global metrics
print("\nGlobal Metrics Across All Series:")
print(f"  Precision: {global_precision:.4f}, Recall: {global_recall:.4f}, F1 Score: {global_f1:.4f}, PR AUC: {global_pr_auc:.4f}")

# Print optimal results for each dataset
for result in optimal_results:
    print(f"\nDataset: {result['dataset']}")
    print(f"  Best F1 Score: {result['best_f1']:.4f}")
    print(f"  Best Window Size: {result['best_window_size']}")
    print(f"  Best Threshold: {result['best_threshold']}")
    print(f"  Precision: {result['precision']:.4f}")
    print(f"  Recall: {result['recall']:.4f}")
    print(f"  PR AUC: {result['pr_auc']:.4f}")

# Extract all F1 scores into an array
f1_scores = [result['best_f1'] for result in optimal_results]

# Print each F1 score formatted individually
formatted_f1_scores = [f"{score:.4f}" for score in f1_scores]
print(f"All F1 Scores for each dataset: {', '.join(formatted_f1_scores)}")





Global Metrics Across All Series:
  Precision: 0.4147, Recall: 0.8263, F1 Score: 0.4761, PR AUC: 0.3645

Dataset: 1
  Best F1 Score: 0.2353
  Best Window Size: 10
  Best Threshold: 99
  Precision: 0.1333
  Recall: 1.0000
  PR AUC: 0.1333

Dataset: 2
  Best F1 Score: 0.9032
  Best Window Size: 100
  Best Threshold: 99
  Precision: 0.9333
  Recall: 0.8750
  PR AUC: 0.8181

Dataset: 3
  Best F1 Score: 1.0000
  Best Window Size: 75
  Best Threshold: 99
  Precision: 1.0000
  Recall: 1.0000
  PR AUC: 1.0000

Dataset: 8
  Best F1 Score: 0.3200
  Best Window Size: 10
  Best Threshold: 99
  Precision: 0.2667
  Recall: 0.4000
  PR AUC: 0.1109

Dataset: 9
  Best F1 Score: 0.1975
  Best Window Size: 10
  Best Threshold: 95
  Precision: 0.1096
  Recall: 1.0000
  PR AUC: 0.1096

Dataset: 10
  Best F1 Score: 0.9286
  Best Window Size: 10
  Best Threshold: 99
  Precision: 0.8667
  Recall: 1.0000
  PR AUC: 0.8667

Dataset: 13
  Best F1 Score: 0.3704
  Best Window Size: 100
  Best Threshold: 99
  Preci

### Yahoo A1B

In [ ]:
from itertools import product
import numpy as np
import pandas as pd
from itertools import product
import warnings
from sklearn.metrics import precision_score, recall_score, f1_score, average_precision_score

# Suppress warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

A1B = [7,20,31,32,48,57,61]

# Initialize global metrics
global_pr_auc_scores = []

# Define hyperparameter ranges
window_sizes = [10, 24, 50, 75, 100, 150]  # Example values; modify as needed
thresholds = [80, 85, 90, 95, 99]  # Percentile thresholds

optimal_results = []

for i in A1B:
    normal_data = dfsA1[i]
    values = normal_data[f"value_{i}"].values
    ground_truth = normal_data['label'].values  # 1 = anomaly, 0 = normal

    best_f1 = 0
    best_params = None
    best_metrics = None

    for window_size, threshold in product(window_sizes, thresholds):
        matrix_profile_distances, score, precision, recall, f1, pr_auc = run_matrix_profile_anomaly_detection_yahoo(
            dfsA1[i],
            f"value_{i}",
            window_size=window_size,
            normalize_data=False,
            smooth_n=(5*30),
            plot_results=False,
            threshold_number=threshold,
        )

        if f1 > best_f1:
            best_f1 = f1
            best_params = (window_size, threshold)
            best_metrics = (precision, recall, pr_auc)

    # Update global metrics with best metrics
    if best_metrics:
        precision, recall, pr_auc = best_metrics
        global_pr_auc_scores.append(pr_auc)

        # Store optimal results for the dataset
        optimal_results.append({
            'dataset': i,
            'best_f1': best_f1,
            'best_window_size': best_params[0],
            'best_threshold': best_params[1],
            'precision': precision,
            'recall': recall,
            'pr_auc': pr_auc
        })

        matrix_profile_distances, score, precision, recall, f1, pr_auc = run_matrix_profile_anomaly_detection_yahoo(
            dfsA1[i],
            f"value_{i}",
            window_size=best_params[0],
            normalize_data=False,
            smooth_n=(5*30),
            plot_results=True,
            threshold_number=best_params[1],
        )

# Compute global metrics
global_recall = np.mean([result['recall'] for result in optimal_results])
global_precision = np.mean([result['precision'] for result in optimal_results])
global_f1 = np.mean([result['best_f1'] for result in optimal_results])
global_pr_auc = np.mean(global_pr_auc_scores)

# Print global metrics
print("\nGlobal Metrics Across All Series:")
print(f"  Precision: {global_precision:.4f}, Recall: {global_recall:.4f}, F1 Score: {global_f1:.4f}, PR AUC: {global_pr_auc:.4f}")

# Print optimal results for each dataset
for result in optimal_results:
    print(f"\nDataset: {result['dataset']}")
    print(f"  Best F1 Score: {result['best_f1']:.4f}")
    print(f"  Best Window Size: {result['best_window_size']}")
    print(f"  Best Threshold: {result['best_threshold']}")
    print(f"  Precision: {result['precision']:.4f}")
    print(f"  Recall: {result['recall']:.4f}")
    print(f"  PR AUC: {result['pr_auc']:.4f}")

# Extract all F1 scores into an array
f1_scores = [result['best_f1'] for result in optimal_results]

# Print each F1 score formatted individually
formatted_f1_scores = [f"{score:.4f}" for score in f1_scores]
print(f"All F1 Scores for each dataset: {', '.join(formatted_f1_scores)}")





Global Metrics Across All Series:
  Precision: 0.4865, Recall: 0.5244, F1 Score: 0.4689, PR AUC: 0.3268

Dataset: 7
  Best F1 Score: 0.6767
  Best Window Size: 24
  Best Threshold: 95
  Precision: 0.6250
  Recall: 0.7377
  PR AUC: 0.4723

Dataset: 20
  Best F1 Score: 0.2857
  Best Window Size: 24
  Best Threshold: 95
  Precision: 0.2083
  Recall: 0.4545
  PR AUC: 0.1074

Dataset: 31
  Best F1 Score: 0.7692
  Best Window Size: 24
  Best Threshold: 99
  Precision: 1.0000
  Recall: 0.6250
  PR AUC: 0.6313

Dataset: 32
  Best F1 Score: 0.7731
  Best Window Size: 24
  Best Threshold: 95
  Precision: 0.6389
  Recall: 0.9787
  PR AUC: 0.6260

Dataset: 48
  Best F1 Score: 0.6154
  Best Window Size: 75
  Best Threshold: 99
  Precision: 0.8000
  Recall: 0.5000
  PR AUC: 0.4083

Dataset: 57
  Best F1 Score: 0.1111
  Best Window Size: 10
  Best Threshold: 99
  Precision: 0.0667
  Recall: 0.3333
  PR AUC: 0.0236

Dataset: 61
  Best F1 Score: 0.0513
  Best Window Size: 10
  Best Threshold: 99
  Pre

### Yahoo A1C

In [ ]:
from itertools import product
import numpy as np
import pandas as pd
from itertools import product
import warnings
from sklearn.metrics import precision_score, recall_score, f1_score, average_precision_score

# Suppress warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

A1C = [17, 19, 26, 37, 40, 47]

# Initialize global metrics
global_pr_auc_scores = []

# Define hyperparameter ranges
window_sizes = [10, 24, 50, 75, 100, 150]  # Example values; modify as needed
thresholds = [80, 85, 90, 95, 99]  # Percentile thresholds

optimal_results = []

for i in A1C:
    normal_data = dfsA1[i]
    values = normal_data[f"value_{i}"].values
    ground_truth = normal_data['label'].values  # 1 = anomaly, 0 = normal

    best_f1 = 0
    best_params = None
    best_metrics = None

    for window_size, threshold in product(window_sizes, thresholds):
        matrix_profile_distances, score, precision, recall, f1, pr_auc = run_matrix_profile_anomaly_detection_yahoo(
            dfsA1[i],
            f"value_{i}",
            window_size=window_size,
            normalize_data=False,
            smooth_n=(5*30),
            plot_results=False,
            threshold_number=threshold,
        )

        if f1 > best_f1:
            best_f1 = f1
            best_params = (window_size, threshold)
            best_metrics = (precision, recall, pr_auc)

    # Update global metrics with best metrics
    if best_metrics:
        precision, recall, pr_auc = best_metrics
        global_pr_auc_scores.append(pr_auc)

        # Store optimal results for the dataset
        optimal_results.append({
            'dataset': i,
            'best_f1': best_f1,
            'best_window_size': best_params[0],
            'best_threshold': best_params[1],
            'precision': precision,
            'recall': recall,
            'pr_auc': pr_auc
        })

        matrix_profile_distances, score, precision, recall, f1, pr_auc = run_matrix_profile_anomaly_detection_yahoo(
            dfsA1[i],
            f"value_{i}",
            window_size=best_params[0],
            normalize_data=False,
            smooth_n=(5*30),
            plot_results=True,
            threshold_number=best_params[1],
        )

# Compute global metrics
global_recall = np.mean([result['recall'] for result in optimal_results])
global_precision = np.mean([result['precision'] for result in optimal_results])
global_f1 = np.mean([result['best_f1'] for result in optimal_results])
global_pr_auc = np.mean(global_pr_auc_scores)

# Print global metrics
print("\nGlobal Metrics Across All Series:")
print(f"  Precision: {global_precision:.4f}, Recall: {global_recall:.4f}, F1 Score: {global_f1:.4f}, PR AUC: {global_pr_auc:.4f}")

# Print optimal results for each dataset
for result in optimal_results:
    print(f"\nDataset: {result['dataset']}")
    print(f"  Best F1 Score: {result['best_f1']:.4f}")
    print(f"  Best Window Size: {result['best_window_size']}")
    print(f"  Best Threshold: {result['best_threshold']}")
    print(f"  Precision: {result['precision']:.4f}")
    print(f"  Recall: {result['recall']:.4f}")
    print(f"  PR AUC: {result['pr_auc']:.4f}")

# Extract all F1 scores into an array
f1_scores = [result['best_f1'] for result in optimal_results]

# Print each F1 score formatted individually
formatted_f1_scores = [f"{score:.4f}" for score in f1_scores]
print(f"All F1 Scores for each dataset: {', '.join(formatted_f1_scores)}")





Global Metrics Across All Series:
  Precision: 0.4130, Recall: 0.6102, F1 Score: 0.4576, PR AUC: 0.3716

Dataset: 17
  Best F1 Score: 0.8980
  Best Window Size: 10
  Best Threshold: 85
  Precision: 0.9252
  Recall: 0.8722
  PR AUC: 0.8274

Dataset: 19
  Best F1 Score: 0.8767
  Best Window Size: 10
  Best Threshold: 80
  Precision: 0.7887
  Recall: 0.9868
  PR AUC: 0.7804

Dataset: 37
  Best F1 Score: 0.3396
  Best Window Size: 24
  Best Threshold: 95
  Precision: 0.2500
  Recall: 0.5294
  PR AUC: 0.1435

Dataset: 40
  Best F1 Score: 0.0273
  Best Window Size: 10
  Best Threshold: 80
  Precision: 0.0175
  Recall: 0.0625
  PR AUC: 0.0537

Dataset: 47
  Best F1 Score: 0.1463
  Best Window Size: 10
  Best Threshold: 95
  Precision: 0.0833
  Recall: 0.6000
  PR AUC: 0.0528
All F1 Scores for each dataset: 0.8980, 0.8767, 0.3396, 0.0273, 0.1463


In [ ]:
matrix_profile_distances, score, precision, recall, f1, pr_auc = run_matrix_profile_anomaly_detection_yahoo(
            dfsA1[26],
            f"value_{26}",
            window_size=120,
            normalize_data=False,
            smooth_n=(5*30),
            plot_results=True,
            threshold_number=95,
        )

In [ ]:
from itertools import product
import numpy as np
import pandas as pd
from itertools import product
import warnings
from sklearn.metrics import precision_score, recall_score, f1_score, average_precision_score

# Suppress warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

A1C = [26]

# Initialize global metrics
global_pr_auc_scores = []

# Define hyperparameter ranges
window_sizes = [10, 24, 50, 75, 100, 150]  # Example values; modify as needed
thresholds = [80, 85, 90, 95, 99]  # Percentile thresholds

optimal_results = []

for i in A1C:
    normal_data = dfsA1[i]
    values = normal_data[f"value_{i}"].values
    ground_truth = normal_data['label'].values  # 1 = anomaly, 0 = normal

    best_f1 = 0
    best_params = None
    best_metrics = None

    for window_size, threshold in product(window_sizes, thresholds):
        matrix_profile_distances, score, precision, recall, f1, pr_auc = run_matrix_profile_anomaly_detection_yahoo(
            dfsA1[i],
            f"value_{i}",
            window_size=window_size,
            normalize_data=False,
            smooth_n=(5*30),
            plot_results=False,
            threshold_number=threshold,
        )

        if f1 > best_f1:
            best_f1 = f1
            best_params = (window_size, threshold)
            best_metrics = (precision, recall, pr_auc)

    # Update global metrics with best metrics
    if best_metrics:
        precision, recall, pr_auc = best_metrics
        global_pr_auc_scores.append(pr_auc)

        # Store optimal results for the dataset
        optimal_results.append({
            'dataset': i,
            'best_f1': best_f1,
            'best_window_size': best_params[0],
            'best_threshold': best_params[1],
            'precision': precision,
            'recall': recall,
            'pr_auc': pr_auc
        })

        matrix_profile_distances, score, precision, recall, f1, pr_auc = run_matrix_profile_anomaly_detection_yahoo(
            dfsA1[i],
            f"value_{i}",
            window_size=best_params[0],
            normalize_data=False,
            smooth_n=(5*30),
            plot_results=True,
            threshold_number=best_params[1],
        )

# Compute global metrics
global_recall = np.mean([result['recall'] for result in optimal_results])
global_precision = np.mean([result['precision'] for result in optimal_results])
global_f1 = np.mean([result['best_f1'] for result in optimal_results])
global_pr_auc = np.mean(global_pr_auc_scores)

# Print global metrics
print("\nGlobal Metrics Across All Series:")
print(f"  Precision: {global_precision:.4f}, Recall: {global_recall:.4f}, F1 Score: {global_f1:.4f}, PR AUC: {global_pr_auc:.4f}")

# Print optimal results for each dataset
for result in optimal_results:
    print(f"\nDataset: {result['dataset']}")
    print(f"  Best F1 Score: {result['best_f1']:.4f}")
    print(f"  Best Window Size: {result['best_window_size']}")
    print(f"  Best Threshold: {result['best_threshold']}")
    print(f"  Precision: {result['precision']:.4f}")
    print(f"  Recall: {result['recall']:.4f}")
    print(f"  PR AUC: {result['pr_auc']:.4f}")

# Extract all F1 scores into an array
f1_scores = [result['best_f1'] for result in optimal_results]

# Print each F1 score formatted individually
formatted_f1_scores = [f"{score:.4f}" for score in f1_scores]
print(f"All F1 Scores for each dataset: {', '.join(formatted_f1_scores)}")





Global Metrics Across All Series:
  Precision: nan, Recall: nan, F1 Score: nan, PR AUC: nan
All F1 Scores for each dataset: 


### Yahoo A1D

In [ ]:
from itertools import product
import numpy as np
import pandas as pd
from itertools import product
import warnings
from sklearn.metrics import precision_score, recall_score, f1_score, average_precision_score

# Suppress warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

A1D = [28,43,46,55,56,65]

# Initialize global metrics
global_pr_auc_scores = []

# Define hyperparameter ranges
window_sizes = [10, 24, 50, 75, 100, 150, 300, 500]  # Example values; modify as needed
thresholds = [80, 85, 90, 95, 99]  # Percentile thresholds

optimal_results = []

for i in A1D:
    normal_data = dfsA1[i]
    values = normal_data[f"value_{i}"].values
    ground_truth = normal_data['label'].values  # 1 = anomaly, 0 = normal

    best_f1 = 0
    best_params = None
    best_metrics = None

    for window_size, threshold in product(window_sizes, thresholds):
        matrix_profile_distances, score, precision, recall, f1, pr_auc = run_matrix_profile_anomaly_detection_yahoo(
            dfsA1[i],
            f"value_{i}",
            window_size=window_size,
            normalize_data=False,
            smooth_n=(5*30),
            plot_results=False,
            threshold_number=threshold,
        )

        if f1 > best_f1:
            best_f1 = f1
            best_params = (window_size, threshold)
            best_metrics = (precision, recall, pr_auc)

    # Update global metrics with best metrics
    if best_metrics:
        precision, recall, pr_auc = best_metrics
        global_pr_auc_scores.append(pr_auc)

        # Store optimal results for the dataset
        optimal_results.append({
            'dataset': i,
            'best_f1': best_f1,
            'best_window_size': best_params[0],
            'best_threshold': best_params[1],
            'precision': precision,
            'recall': recall,
            'pr_auc': pr_auc
        })

        matrix_profile_distances, score, precision, recall, f1, pr_auc = run_matrix_profile_anomaly_detection_yahoo(
            dfsA1[i],
            f"value_{i}",
            window_size=best_params[0],
            normalize_data=False,
            smooth_n=(5*30),
            plot_results=True,
            threshold_number=best_params[1],
        )

# Compute global metrics
global_recall = np.mean([result['recall'] for result in optimal_results])
global_precision = np.mean([result['precision'] for result in optimal_results])
global_f1 = np.mean([result['best_f1'] for result in optimal_results])
global_pr_auc = np.mean(global_pr_auc_scores)

# Print global metrics
print("\nGlobal Metrics Across All Series:")
print(f"  Precision: {global_precision:.4f}, Recall: {global_recall:.4f}, F1 Score: {global_f1:.4f}, PR AUC: {global_pr_auc:.4f}")

# Print optimal results for each dataset
for result in optimal_results:
    print(f"\nDataset: {result['dataset']}")
    print(f"  Best F1 Score: {result['best_f1']:.4f}")
    print(f"  Best Window Size: {result['best_window_size']}")
    print(f"  Best Threshold: {result['best_threshold']}")
    print(f"  Precision: {result['precision']:.4f}")
    print(f"  Recall: {result['recall']:.4f}")
    print(f"  PR AUC: {result['pr_auc']:.4f}")

# Extract all F1 scores into an array
f1_scores = [result['best_f1'] for result in optimal_results]

# Print each F1 score formatted individually
formatted_f1_scores = [f"{score:.4f}" for score in f1_scores]
print(f"All F1 Scores for each dataset: {', '.join(formatted_f1_scores)}")





Global Metrics Across All Series:
  Precision: 0.1412, Recall: 0.6543, F1 Score: 0.2159, PR AUC: 0.1257

Dataset: 28
  Best F1 Score: 0.1081
  Best Window Size: 75
  Best Threshold: 80
  Precision: 0.0694
  Recall: 0.2439
  PR AUC: 0.0600

Dataset: 43
  Best F1 Score: 0.5385
  Best Window Size: 24
  Best Threshold: 95
  Precision: 0.3889
  Recall: 0.8750
  PR AUC: 0.3431

Dataset: 46
  Best F1 Score: 0.1008
  Best Window Size: 50
  Best Threshold: 80
  Precision: 0.0694
  Recall: 0.1835
  PR AUC: 0.0745

Dataset: 55
  Best F1 Score: 0.1039
  Best Window Size: 10
  Best Threshold: 95
  Precision: 0.0556
  Recall: 0.8000
  PR AUC: 0.0451

Dataset: 56
  Best F1 Score: 0.1299
  Best Window Size: 10
  Best Threshold: 95
  Precision: 0.0694
  Recall: 1.0000
  PR AUC: 0.0694

Dataset: 65
  Best F1 Score: 0.3146
  Best Window Size: 24
  Best Threshold: 95
  Precision: 0.1944
  Recall: 0.8235
  PR AUC: 0.1622
All F1 Scores for each dataset: 0.1081, 0.5385, 0.1008, 0.1039, 0.1299, 0.3146


### NAB

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)


data = {}
path = '/content/drive/My Drive/NAB/data'

def addFolderAndReadAll(f_name):
    data[f_name] = {}
    csvs = os.listdir(path + '/' + f_name)
    csvs_num = 0
    for csv in csvs:
        data[f_name][csv] = pd.read_csv(path + '/' + f_name + '/' + csv)
        csvs_num += 1
    return csvs_num

csvs_num = sum([addFolderAndReadAll(f_name) for f_name in os.listdir(path)])

with open('/content/drive//My Drive/NAB/labels/combined_windows.json', 'r') as file:
    labels = json.load(file)

for key1 in data:
    for key2 in data[key1]:
        one_csv_labels = labels[key1+'/'+key2]
        data[key1][key2]['label'] = np.array(0)
        num = 0
        for interval in one_csv_labels:
            #print(key2, interval, num)
            index1 = list(data[key1][key2]['timestamp']).index(interval[0][:19])
            index2 = list(data[key1][key2]['timestamp']).index(interval[1][:19])
            data[key1][key2]['label'][num] = slice(index1, index2)
            #data[key1][key2]['label'] = data[key1][key2]['label'].apply(lambda x: 1 if isinstance(x, slice) else 0)
            num += 1

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)


data = {}
path = '/content/drive/My Drive/NAB/data'

def addFolderAndReadAll(f_name):
    data[f_name] = {}
    csvs = os.listdir(path + '/' + f_name)
    csvs_num = 0
    for csv in csvs:
        data[f_name][csv] = pd.read_csv(path + '/' + f_name + '/' + csv)
        csvs_num += 1
    return csvs_num

csvs_num = sum([addFolderAndReadAll(f_name) for f_name in os.listdir(path)])

with open('/content/drive//My Drive/NAB/labels/combined_windows.json', 'r') as file:
    labels = json.load(file)

for key1 in data:
    for key2 in data[key1]:
        one_csv_labels = labels[key1+'/'+key2]
        data[key1][key2]['label'] = np.array(0)
        num = 0
        for interval in one_csv_labels:
            #print(key2, interval, num)
            index1 = list(data[key1][key2]['timestamp']).index(interval[0][:19])
            index2 = list(data[key1][key2]['timestamp']).index(interval[1][:19])
            data[key1][key2]['label'][num] = slice(index1, index2)
            #data[key1][key2]['label'] = data[key1][key2]['label'].apply(lambda x: 1 if isinstance(x, slice) else 0)
            num += 1

import os
import pandas as pd
import numpy as np
import json

data = {}
path = '/content/drive/My Drive/NAB/data'

# Load all CSV files into a nested dictionary
def addFolderAndReadAll(f_name):
    data[f_name] = {}
    csvs = os.listdir(path + '/' + f_name)
    csvs_num = 0
    for csv in csvs:
        data[f_name][csv] = pd.read_csv(path + '/' + f_name + '/' + csv)
        csvs_num += 1
    return csvs_num

# Read data from all folders
csvs_num = sum([addFolderAndReadAll(f_name) for f_name in os.listdir(path)])

# Load anomaly intervals from JSON
with open('/content/drive/My Drive/NAB/labels/combined_windows.json', 'r') as file:
    labels = json.load(file)

# Process each CSV and assign binary labels
for key1 in data:
    for key2 in data[key1]:
        one_csv_labels = labels[key1 + '/' + key2]
        df = data[key1][key2]
        df['label'] = 0  # Initialize label column with 0

        # Mark intervals as anomalies
        for interval in one_csv_labels:
            index1 = list(df['timestamp']).index(interval[0][:19])
            index2 = list(df['timestamp']).index(interval[1][:19])
            df.loc[index1:index2, 'label'] = 1  # Set rows within interval to 1

        # Update the DataFrame back into the data dictionary
        data[key1][key2] = df

# All `data` dictionaries now have binary labels.

In [ ]:
import math

def sigmoid(x):
    """Standard sigmoid function."""
    return 1 / (1 + math.exp(-x))

def sigmoid_score(t, A_tp, A_fp):
    """
    Calculate the scaled sigmoid score for a detection.
    @param t: Relative position of the detection:
              - t = -1 is far left of the window, t = 0 is the right edge.
              - t > 0 considers detections outside the window.
    @param A_tp: Reward for true positives.
    @param A_fp: Penalty for false positives.
    """
    if t > 3.0:
        return -1.0  # Assign -1 for detections too far after the window
    elif 0 < t <= 3.0:
        return  2* (1 / (1 + math.exp(5 * t))) - 1.0  # Sigmoid scaling
    else:
        return  2* (1 / (1 + math.exp(5 * t))) - 1.0  # Normal sigmoid for in-window cases

def compute_raw_score_debug(detections, anomaly_windows, A_tp, A_fp, A_fn):
    """
    Compute the raw NAB score for a dataset with detailed debugging.
    Also computes S_perfect and S_null for comparison.
    @param detections: List of detected anomaly timestamps.
    @param anomaly_windows: List of anomaly windows (start_time, end_time).
    @param A_tp: Reward for true positives.
    @param A_fp: Penalty for false positives.
    @param A_fn: Penalty for missed anomaly windows (false negatives).

    @return: Raw NAB score, S_perfect, S_null.
    """
    score = 0
    missed_windows = 0


    # Sort detections and anomaly windows
    detections = sorted(detections)
    anomaly_windows = sorted(anomaly_windows, key=lambda x: x[0])

    # Compute the actual score
    for i, window in enumerate(anomaly_windows):
        start_time, end_time = window
        window_length = end_time - start_time

        # Find detections within or after the current window
        detections_in_window = [d for d in detections if d >= start_time]

        if detections_in_window and start_time <= detections_in_window[0] <= end_time:
            # Detection within the window
            first_detection = detections_in_window[0]
            t = -3 + 2 * (first_detection - start_time) / window_length
            contribution = sigmoid_score(t, A_tp, A_fp)
            print(f"Window {i}: {window}, Detection t={t:.3f}, Contribution={contribution:.3f}")
            score += contribution
            # Remove the detection
            detections.remove(first_detection)
            detections_in_window = [d for d in detections if start_time <= d <= end_time]
            for d in detections_in_window:
                detections.remove(d)


        elif detections_in_window:
            # Check all detections just outside the current window
            near_miss_detections = [d for d in detections_in_window if 0 < (d - end_time) <= 3.0]

            for near_detection in near_miss_detections:
                t = (near_detection - end_time)  # Relative position after the window
                contribution = sigmoid_score(t, A_tp, A_fp)
                print(f"Window {i}: {window}, Near-miss detection t={t:.3f}, Contribution={contribution:.3f}")
                score += contribution
                # Remove the detection after scoring it
                detections.remove(near_detection)
        else:
            # No detections in this window - count as a false negative
            print(f"Window {i}: {window}, Missed anomaly window. Penalty={A_fn:.3f}")
            score += A_fn
            missed_windows += 1

    # Handle remaining detections as false positives or outside windows
    for detection in detections:
        # Check if detection is close to the end of any window
        near_miss_found = False
        for window in anomaly_windows:
            _, end_time = window
            if 0 < (detection - end_time) <= 3.0:
                t = (detection - end_time)
                contribution = sigmoid_score(t, A_tp, A_fp)
                print(f"Detection {detection} near-miss outside any window. t={t:.3f}, Contribution={contribution:.3f}")
                score += contribution
                near_miss_found = True
                break

        if not near_miss_found:
            # Treat as a false positive
            contribution = sigmoid_score(3.0, A_tp, A_fp)  # Use t=3.0 for far detections
            print(f"False positive detection at {detection}, Contribution={contribution:.3f}")
            score += contribution

    print(f"Raw score for dataset: {score:.3f}")

    return score


In [ ]:
def create_anomaly_windows(binary_labels):
    """
    Generate an array of anomaly windows (start, end) tuples from binary labels.

    @param binary_labels: List or array of binary labels (0s and 1s).
                          1 indicates an anomaly, 0 indicates normal.

    @return: List of tuples where each tuple is (start_index, end_index) for an anomaly window.
    """
    anomaly_windows = []
    in_window = False
    start_index = None

    for i, label in enumerate(binary_labels):
        if label == 1 and not in_window:
            # Start of a new anomaly window
            start_index = i
            in_window = True
        elif label == 0 and in_window:
            # End of the current anomaly window
            end_index = i - 1
            anomaly_windows.append((start_index, end_index))
            in_window = False

    # If the last label is part of an anomaly window, close it
    if in_window:
        anomaly_windows.append((start_index, len(binary_labels) - 1))

    return anomaly_windows


In [ ]:
anomaly_windows = {}
binary_labels = data["realKnownCause"]["nyc_taxi.csv"]["label"]
# Generate the anomaly windows for the current labels
anomaly_windows["nyc_taxi.csv"] = create_anomaly_windows(binary_labels)

In [ ]:
from itertools import product
import warnings
import numpy as np
from sklearn.metrics import precision_score, recall_score
import pandas as pd
import stumpy  # Assuming matrix profile library


warnings.filterwarnings("ignore", category=RuntimeWarning)


def run_matrix_profile_anomaly_detection_nab(df, column_name: str, window_size: int = 96, smooth_n: int = 10, normalize_data: bool = True, threshold_numbers=None):
    """
    df: Pandas DataFrame
    column_name: Name of the column to analyze
    window_size: Number of data points in a pattern
    smooth_n: Smoothing window for matrix profile scores
    normalize_data: Normalize column data
    threshold_numbers: List of percentile thresholds to test

    Returns:
    Best NAB score, precision, recall, and associated threshold
    """
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in the DataFrame.")

    # Select the specified column
    column_data = df[column_name].values

    # Normalize data if required
    if normalize_data:
        column_data = (column_data - np.min(column_data)) / (np.max(column_data) - np.min(column_data))

    # Calculate matrix profile
    mp = stumpy.stump(column_data.astype("float"), m=window_size, ignore_trivial=True, normalize=False)
    mp_dist = mp[:, 0]

    # Smooth the matrix profile scores
    df_mp_dist = pd.DataFrame(mp_dist).rolling(smooth_n).mean()

    # Fill NaN values
    nan_value_count = np.empty(len(column_data) - len(mp_dist))
    nan_value_count.fill(np.nan)
    mp_dist = np.concatenate((nan_value_count, mp_dist.astype(float)))

    # Remove NaN values and replace them with 0
    mp_dist[np.isnan(mp_dist)] = 0

    # Set thresholds
    if threshold_numbers is None:
        threshold_numbers = [80, 85, 90, 95, 99]  # Default percentiles

    best_nab = -np.inf
    best_precision = 0
    best_recall = 0
    best_threshold = None

    for threshold_number in threshold_numbers:
        # Apply the threshold
        threshold = np.percentile(mp_dist, threshold_number)
        pred_anomalies = (mp_dist > threshold).astype(int)

        # Convert binary predictions to detection timestamps
        detections = [i for i, x in enumerate(pred_anomalies) if x == 1]

        # Compute the NAB score
        nab_score = compute_raw_score_debug(
            detections, anomaly_windows["nyc_taxi.csv"], A_tp=1.0, A_fp=-0.11, A_fn=-1.0
        )

        # Evaluate precision and recall
        ground_truth = df['label'].values
        precision = precision_score(ground_truth, pred_anomalies)
        recall = recall_score(ground_truth, pred_anomalies)

        # Update the best parameters
        if nab_score > best_nab:
            best_nab = nab_score
            best_precision = precision
            best_recall = recall
            best_threshold = threshold_number

    print(f"Best NAB Score: {best_nab:.4f}")
    print(f"Precision: {best_precision:.4f}, Recall: {best_recall:.4f}")
    print(f"Best Threshold Percentile: {best_threshold}%")

    return best_nab, best_precision, best_recall, best_threshold


In [ ]:
from itertools import product
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score, recall_score
import stumpy

def optimize_matrix_profile_nab(df, column_name: str, window_sizes: list, thresholds: list, smooth_n: int = 10, normalize_data: bool = True):
    """
    Optimize Matrix Profile anomaly detection by finding the best window size and threshold.

    Args:
    - df: Pandas DataFrame containing the time series data and labels.
    - column_name: Name of the column with time series values.
    - window_sizes: List of window sizes to try.
    - thresholds: List of percentile thresholds to try.
    - smooth_n: Smoothing window for matrix profile scores.
    - normalize_data: Whether to normalize the time series values.

    Returns:
    - Optimal NAB score, precision, recall, best window size, and best threshold.
    """
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in the DataFrame.")

    column_data = df[column_name].values
    ground_truth = df['label'].values  # Assuming 'label' column exists

    if normalize_data:
        column_data = (column_data - np.min(column_data)) / (np.max(column_data) - np.min(column_data))

    best_nab = -np.inf
    best_precision = 0
    best_recall = 0
    best_window_size = None
    best_threshold = None

    for window_size, threshold_number in product(window_sizes, thresholds):
        try:
            # Calculate matrix profile
            mp = stumpy.stump(column_data.astype("float"), m=window_size, ignore_trivial=True, normalize=False)
            mp_dist = mp[:, 0]

            # Smooth matrix profile scores
            df_mp_dist = pd.DataFrame(mp_dist).rolling(smooth_n).mean()

            # Fill NaN values
            nan_value_count = np.empty(len(column_data) - len(mp_dist))
            nan_value_count.fill(np.nan)
            mp_dist = np.concatenate((nan_value_count, mp_dist.astype(float)))
            mp_dist[np.isnan(mp_dist)] = 0

            # Apply threshold
            threshold = np.percentile(mp_dist, threshold_number)
            pred_anomalies = (mp_dist > threshold).astype(int)

            # Convert binary predictions to detection timestamps
            detections = [i for i, x in enumerate(pred_anomalies) if x == 1]

            # Compute NAB score
            nab_score = compute_raw_score_debug(
                detections, anomaly_windows["nyc_taxi.csv"], A_tp=1.0, A_fp=-0.11, A_fn=-1.0
            )

            # Evaluate precision and recall
            precision = precision_score(ground_truth, pred_anomalies)
            recall = recall_score(ground_truth, pred_anomalies)

            # Update best parameters
            if nab_score > best_nab:
                best_nab = nab_score
                best_precision = precision
                best_recall = recall
                best_window_size = window_size
                best_threshold = threshold_number

        except Exception as e:
            print(f"Error with window_size={window_size} and threshold={threshold_number}: {e}")

    print(f"Optimal NAB Score: {best_nab:.4f}")
    print(f"Precision: {best_precision:.4f}, Recall: {best_recall:.4f}")
    print(f"Best Window Size: {best_window_size}, Best Threshold Percentile: {best_threshold}")

    return best_nab, best_precision, best_recall, best_window_size, best_threshold


In [ ]:
# Example inputs
window_sizes = [48, 2 * 48, 3 * 48, 4 * 48, 5 * 48]  # Example window sizes
thresholds = [80, 85, 90, 95, 99]  # Percentile thresholds

# Optimize for the NYC Taxi dataset
best_nab, best_precision, best_recall, best_window_size, best_threshold = optimize_matrix_profile_nab(
    data["realKnownCause"]["nyc_taxi.csv"],
    column_name="value",
    window_sizes=window_sizes,
    thresholds=thresholds,
    smooth_n=10,
    normalize_data=True
)

print(f"Optimal Results:")
print(f"  NAB Score: {best_nab}")
print(f"  Precision: {best_precision}")
print(f"  Recall: {best_recall}")
print(f"  Best Window Size: {best_window_size}")
print(f"  Best Threshold Percentile: {best_threshold}")


Η έξοδος ροής περικόπηκε στις τελευταίες 5000 γραμμές.
False positive detection at 10242, Contribution=-1.000
False positive detection at 10243, Contribution=-1.000
False positive detection at 10244, Contribution=-1.000
False positive detection at 10245, Contribution=-1.000
False positive detection at 10246, Contribution=-1.000
False positive detection at 10247, Contribution=-1.000
False positive detection at 10248, Contribution=-1.000
False positive detection at 10249, Contribution=-1.000
False positive detection at 10250, Contribution=-1.000
False positive detection at 10251, Contribution=-1.000
False positive detection at 10252, Contribution=-1.000
False positive detection at 10253, Contribution=-1.000
False positive detection at 10254, Contribution=-1.000
False positive detection at 10255, Contribution=-1.000
False positive detection at 10256, Contribution=-1.000
False positive detection at 10257, Contribution=-1.000
False positive detection at 10258, Contribution=-1.000
False posi

In [ ]:
def run_matrix_profile_anomaly_detection_yahoo(df, column_name: str, window_size: int = 96, smooth_n: int = 10, normalize_data: bool = True, plot_results: bool = True, threshold_number=95):
    """
    df: Pandas DataFrame
    column_name: Name of the column to analyze
    window_size: Approximately, how many data points might be found in a pattern
    smooth_n: Time windows to smooth matrix profile scores by
    normalize_data: Normalize dataframe or not. Advisable when using columns which are in diff. scales. Boolean
    plot_results: Return a plotly plot of the results. Includes the original timeseries and the matrix profile.
    threshold_number: Percentile threshold for anomaly detection

    Returns:
    Matrix Profile Scores, ranking score for the column, and evaluation metrics (precision, recall, F1-score, PR AUC).
    """
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in the DataFrame.")

    # Select the specified column
    column_data = df[column_name]

    # Normalize data if required
    #if normalize_data:
    #    column_data = (normalize(column_data).dropna())

    # Calculate matrix profile
    mp = stumpy.stump(column_data.astype("float"), m=window_size, ignore_trivial=True, normalize=False)
    mp_dist = mp[:, 0]

    # Smooth the matrix profile scores
    df_mp_dist = pd.DataFrame(mp_dist).rolling(smooth_n).mean()
    score = df_mp_dist[0].quantile(0.995) - df_mp_dist[0].quantile(0.75)

    # Fill NaN values
    nan_value_count = np.empty(len(column_data) - len(mp_dist))
    nan_value_count.fill(np.nan)
    mp_dist = np.concatenate((nan_value_count, mp_dist.astype(float)))

    # Plot the results if required
    if plot_results:
        pd.options.plotting.backend = "plotly"
        df_tmp = df[[column_name]].assign(MatrixProfileScore=mp_dist)
        fig = df_tmp.plot(template="simple_white", title=f"{column_name}")
        fig.show()

    # Evaluate metrics
    ground_truth = df['label'].values

    # Remove NaN values and replace them with 0
    mp_dist[np.isnan(mp_dist)] = 0

    # Apply a threshold for the matrix profile
    threshold = np.percentile(mp_dist, threshold_number)

    # Find the anomalies
    predicted_anomalies = (mp_dist > threshold).astype(int)

    # Convert binary predictions to detection timestamps
    detections = [i for i, x in enumerate(predicted_anomalies) if x == 1]

    # Compute NAB score
    nab_score = compute_raw_score_debug(
        detections, anomaly_windows["nyc_taxi.csv"], A_tp=1.0, A_fp=-0.11, A_fn=-1.0
    )

    # Compute metrics using sklearn
    precision = precision_score(ground_truth, predicted_anomalies)
    recall = recall_score(ground_truth, predicted_anomalies)
    f1 = f1_score(ground_truth, predicted_anomalies)
    pr_auc = average_precision_score(ground_truth, predicted_anomalies)

    return mp_dist, score, precision, recall, f1, pr_auc, nab_score


In [ ]:
run_matrix_profile_anomaly_detection_yahoo(data["realKnownCause"]["nyc_taxi.csv"],column_name="value",window_size=48,smooth_n=10,normalize_data=True,plot_results=True,threshold_number=99)

Window 0: (5839, 6045), Detection t=-1.864, Contribution=1.000
Window 4: (9977, 10183), Detection t=-2.117, Contribution=1.000
Raw score for dataset: 2.000


(array([   0.        ,    0.        ,    0.        , ..., 6606.71938257,
        6628.87298113, 6903.2627793 ]),
 19327.59011964797,
 1.0,
 0.10048309178743961,
 0.1826163301141352,
 0.19069627008201323,
 1.9997701485776767)

In [ ]:
run_matrix_profile_anomaly_detection_yahoo(data["realKnownCause"]["nyc_taxi.csv"],column_name="value",window_size=48,smooth_n=10,normalize_data=True,plot_results=True,threshold_number=98)

Window 0: (5839, 6045), Detection t=-1.883, Contribution=1.000
Window 2: (8423, 8629), Detection t=-1.883, Contribution=1.000
Window 3: (8731, 8937), Detection t=-2.010, Contribution=1.000
Window 4: (9977, 10183), Detection t=-2.146, Contribution=1.000
Raw score for dataset: 4.000


(array([   0.        ,    0.        ,    0.        , ..., 6606.71938257,
        6628.87298113, 6903.2627793 ]),
 19327.59011964797,
 1.0,
 0.2,
 0.3333333333333333,
 0.2802325581395349,
 3.999544531854375)

## MP NAB SCORES: FOR 95% THRESHOLD: -76 SPOTS ALL 5 WINDOWS
##                FOR 96% THRESHOLD: -46 SPOTS ALL 5 WINDOWS
##                FOR 97% THRESHOLD: -5 MISSES 2ND WINDOW
##                FOR 98% THRESHOLD:  4 MISSES 2ND WINDOW
##                FOR 99% THRESHOLD: FINDS ONLY 2 WINDOWS
